# StableOpt for Wake Model with $\theta=0$ and $45$

In [8]:
import time
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy.optimize import Bounds
import scipy.spatial.distance as spdist
%matplotlib inline
try:
  import py_wake
except:
  !pip install py_wake

In [9]:
# Calculate C_T^* as a function of S_x, S_y, theta. Implement wake model in Python.
import numpy as np
from py_wake import YZGrid
from py_wake.turbulence_models import CrespoHernandez
from py_wake.superposition_models import LinearSum
from py_wake.rotor_avg_models import GQGridRotorAvg
from py_wake.wind_farm_models.engineering_models import PropagateDownwind
from py_wake.wind_turbines import OneTypeWindTurbines
from py_wake.deficit_models import NiayifarGaussianDeficit
from py_wake.site import UniformSite

def wake_model(S_x, S_y, theta):
    """Use a low order wake model to
    calculate C_T^*

    :param S_x: float distance between turbines in the x direction (metres)
    :param S_y: float distance between turbines in the y direction (metres)
    :param theta: float wind direction with respect to x direction (degrees)

    :returns ct_star: float local turbine thrust coefficient (dimensionless)
    """

    #estimate thrust coefficient ct
    ct_prime = 1.33
    a = ct_prime/(4 + ct_prime)
    ct = 4*a*(1-a)

    #define a farm site with a background turbulence intensity of 0.1 (10%)
    site = UniformSite([1],0.1)

    #calculate turbine coordinates
    x = np.hstack((np.arange(0, -10000, -S_x),np.arange(S_x, 1500, S_x)))
    y = np.hstack((np.arange(0, 8500, S_y),np.arange(-S_y,-2000,-S_y)))
    xx, yy = np.meshgrid(x,y)
    x = xx.flatten()
    y = yy.flatten()

    #only consider turbines 10km upstream or 1km in the cross stream direction
    streamwise_cond = -x*np.cos(theta*np.pi/180) +y*np.sin(theta*np.pi/180) < 10000
    spanwise_cond = abs(-y*np.cos(theta*np.pi/180) - x*np.sin(theta*np.pi/180)) < 2000
    total_cond = np.logical_and(streamwise_cond, spanwise_cond)
    x_rot = x[total_cond]
    y_rot = y[total_cond]

    #create ideal turbines with constant thrust coefficients
    my_wt = OneTypeWindTurbines(name='MyWT',
                           diameter=100,
                           hub_height=100,
                           ct_func=lambda ws : np.interp(ws, [0, 30], [ct, ct]),
                           power_func=lambda ws : np.interp(ws, [0, 30], [2, 2]),
                           power_unit='kW')
    windTurbines = my_wt

    #select models to calculate wake deficits behind turbines
    wake_deficit = PropagateDownwind(site, windTurbines, NiayifarGaussianDeficit(a=[0.38, 4e-3],use_effective_ws=True),
                                superpositionModel=LinearSum(), rotorAvgModel = GQGridRotorAvg(4,3),
                                turbulenceModel=CrespoHernandez())
    
    #run wind farm simulation
    simulationResult = wake_deficit(x_rot, y_rot, ws=10, wd=270+theta)

    #calculate turbine disk velocity
    U_T = (1-a)*simulationResult.WS_eff[0]

    #calculate velocity in wind farm layer (0-250m above the surface)
    U_F = 0
    for i in np.linspace(-S_x, 0, 200):
        grid = YZGrid(x = i, y = np.linspace(-S_y/2,S_y/2,200),
                        z = np.linspace(0,250,20))
        flow_map = simulationResult.flow_map(grid=grid, ws=10, wd=270+theta)
        U_F += np.mean(flow_map.WS_eff)
    U_F = U_F/200

    #calculate local turbine thrust coefficient
    ct_star = float(ct_prime*(U_T/U_F)**2)
    return ct_star

In [3]:
# wake model with theta=0
def wake_model_0(w):
    '''
    w should be an numpy array or python list of size 2 correspond to the value of
    S_x, S_y in wake_lake function.
    '''
    x,y=w
    return wake_model(x,y,0)

In [4]:
# wake model with pertubation
def wake_model_0_perturbed(w):
    
    def k(theta):
        if np.linalg.norm(theta)<=10 and 500<=w[0]-theta[0]<=1000 and 500<=w[1]-theta[1]<=1000:
            return wake_model_0(w-theta)
        else:
            return np.inf
        
    opt=scipy.optimize.differential_evolution(lambda theta: k(theta),bounds=[(-10,10), (-10,10)])
    return opt.fun

In [53]:
# Time taken for evaluating wake_model_0_perturbed for once.
start=time.time()
t=wake_model_0_perturbed(np.array([900,900]))
end=time.time()
duration=end-start
duration

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


11380.160067796707

In [48]:
# Time take for evaluating wake model for once.
start=time.time()
wake_model_0(np.array([900,1000]))
end=time.time()
duration=end-start
duration

61.48469281196594

In [14]:
# ARD SE kernel
def covSEard(hyp=None, x=None, z=None):
    ''' Squared Exponential covariance function with Automatic Relevance Detemination
     (ARD) distance measure. The covariance function is parameterized as:
    
     k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)
    
     where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
     D is the dimension of the input space and sf2 is the signal variance.
    
     The hyperparameters are:
    
     hyp = [ log(ell_1)
             log(ell_2)
             ...
             log(ell_D)
             log(sqrt(sf2)) ], which is an 1-D list.
             x should be an n*D matrix
             z should be a list, for example [[1,2],[3,4]]
    '''
    
    if hyp == None:                 # report number of parameters
        return ['D + 1']            # USAGE: integer OR D_+_int (spaces are SIGNIFICANT)
    
    [n, D] = x.shape
    ell = 1/np.exp(hyp[0:D])        # characteristic length scale

    sf2 = np.exp(2.*hyp[D])         # signal variance

    if z == 'diag':
        A = np.zeros((n,1))
    elif z == None:
        tmp = np.dot(np.diag(ell),x.T).T
        A = spdist.cdist(tmp, tmp, 'sqeuclidean')
    else:                           # compute covariance between data sets x and z
        A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),np.asarray(z).T).T, 'sqeuclidean') # cross covariances
 
    A = sf2*np.exp(-0.5*A)

                
    return A

In [6]:
# Sampling 50 data 
np.random.seed(41)
points = []
num_points=50

for i in range(num_points):
    x=np.random.uniform(500,1000)
    y=np.random.uniform(500,1000)
    points = points + [[x,y]]

sample_points=np.asarray(points)
sample_funval=np.asarray([wake_model_0(x) for x in sample_points]).reshape(50,1)


In [7]:
# Obtain hyperparameters in of ARD SE Kernel via maximum likelihood of marginal GP log likelihood
def logmarli(x):
    '''
    x is a list of hyperparameters consists of [ell_1 ... ell_D, sqrt(sf2)] 
    and must be positive.
    
    '''
    size=sample_points.shape[0]
    log_x=np.log(x)
    par=log_x.tolist()
    K=covSEard(par,sample_points)
    K_t=K+0.0001*np.identity(size)
    neg_log_marginal_likelihood=0.5*np.dot(np.dot(sample_funval.T,np.linalg.inv(K_t)),sample_funval)+0.5*np.log(np.linalg.det(K_t))+250*np.log(2*np.pi)
    
    return neg_log_marginal_likelihood[0,0]

ard=scipy.optimize.minimize(lambda w:logmarli(w), np.array([2.0,2.0,0.6]),method="L-BFGS-B",options = {'disp':True}) #optimized value of ell_1, ell_2 and sqrt(sf2)
ard_param=np.log(ard.x).tolist()

In [47]:
train_points_1st=sample_points
train_funval_1st=sample_funval
bounds=((500, 1000), (500, 1000))

#set the observation noise and the beta parameter
noise=0.0005
beta_sqrt=4
# define mean function, variance function, lcb, ucb of GP model
def mu(x,input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    n,D=input_data_x.shape
    x_list=x.reshape(1,2).tolist()
    K_t=covSEard(ard_param,input_data_x)+noise*np.identity(n)
    mean=np.dot(np.dot(covSEard(ard_param,input_data_x,x_list).T,np.linalg.inv(K_t)),input_data_y)
    
    return mean[0,0]

def sigma_sq(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    n,D=input_data_x.shape
    x_vec=x.reshape(1,2)
    x_list=x_vec.tolist()
    K_t=covSEard(ard_param,input_data_x)+noise*np.identity(n)
    var=covSEard(ard_param,x_vec,x_list)-np.dot(np.dot(covSEard(ard_param,input_data_x,x_list).T,np.linalg.inv(K_t)),covSEard(ard_param,input_data_x,x_list))
    return var[0,0]

def ucb(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    ucb=mu(x,input_data_x, input_data_y)+beta_sqrt*np.sqrt(sigma_sq(x, input_data_x, input_data_y))
    return ucb

def lcb(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    lcb=mu(x,input_data_x, input_data_y)-beta_sqrt*np.sqrt(sigma_sq(x, input_data_x, input_data_y))
    return lcb

In [48]:
# First 20 iterations of the first experiment for wake model with theta=0 with 50 sampled data points as initial points.
returned_points_1st_exp=[] # returned points in all iterations
T=20 #number of iteration

for t in range(T):
    
    def ucb_min(x):
        
        def k(theta):
            if np.linalg.norm(theta)<=10 and 500<=x[0]-theta[0]<=1000 and 500<=x[1]-theta[1]<=1000:
                return ucb(x-theta,train_points_1st,train_funval_1st)
            else:
                return np.inf
        
        opt=scipy.optimize.differential_evolution(lambda theta: k(theta),bounds=[(-10,10), (-10,10)])
        
        return opt.fun
        
    max_ucb=scipy.optimize.shgo(lambda w:-ucb_min(w), bounds,n=500, iters=5)
    
    x_bar_t=max_ucb.x
    
    def lcb_perturbed(theta):
        
        if np.linalg.norm(theta)<=10 and 500<=x_bar_t[0]-theta[0]<=1000 and 500<=x_bar_t[1]-theta[1]<=1000:
            return lcb(x_bar_t-theta,train_points_1st,train_funval_1st)
        else:
            return np.inf
        
    
    min_lcb=scipy.optimize.differential_evolution(lambda theta: lcb_perturbed(theta),bounds=[(-10,10), (-10,10)])
    
    theta_t=min_lcb.x
    
    y_t=wake_model_0(x_bar_t-theta_t)
    
    train_points_1st=np.row_stack([train_points_1st,x_bar_t-theta_t])
    
    train_funval_1st=np.row_stack([train_funval_1st,y_t])
    
    returned_points_1st_exp=returned_points_1st_exp+[x_bar_t]
    

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df

In [65]:
# update the hyperparameter of the kernel using the first 70 training samples
def logmarli(x):
    '''
    x is a list of hyperparameters consists of [ell_1 ... ell_D, sqrt(sf2)] 
    and must be positive.
    
    '''
    size=train_points_1st.shape[0]
    log_x=np.log(x)
    par=log_x.tolist()
    K=covSEard(par,train_points_1st)
    K_t=K+0.0001*np.identity(size)
    neg_log_marginal_likelihood=0.5*np.dot(np.dot(train_funval_1st.T,np.linalg.inv(K_t)),train_funval_1st)+0.5*np.log(np.linalg.det(K_t))+250*np.log(2*np.pi)
    
    return neg_log_marginal_likelihood[0,0]

ard=scipy.optimize.minimize(lambda w:logmarli(w), np.array([1.0,1.0,0.5]),method="L-BFGS-B",options = {'disp':True}) #optimized value of ell_1, ell_2 and sqrt(sf2)
ard_param=np.log(ard.x).tolist()


In [67]:
# Second 20 iterations of the first experiment
for t in range(T):
    
    def ucb_min(x):
        
        def k(theta):
            if np.linalg.norm(theta)<=10 and 500<=x[0]-theta[0]<=1000 and 500<=x[1]-theta[1]<=1000:
                return ucb(x-theta,train_points_1st,train_funval_1st)
            else:
                return np.inf
        
        opt=scipy.optimize.differential_evolution(lambda theta: k(theta),bounds=[(-10,10), (-10,10)])
        
        return opt.fun
        
    max_ucb=scipy.optimize.shgo(lambda w:-ucb_min(w), bounds,n=500, iters=5)
    
    x_bar_t=max_ucb.x
    
    def lcb_perturbed(theta):
        
        if np.linalg.norm(theta)<=10 and 500<=x_bar_t[0]-theta[0]<=1000 and 500<=x_bar_t[1]-theta[1]<=1000:
            return lcb(x_bar_t-theta,train_points_1st,train_funval_1st)
        else:
            return np.inf
        
    
    min_lcb=scipy.optimize.differential_evolution(lambda theta: lcb_perturbed(theta),bounds=[(-10,10), (-10,10)])
    
    theta_t=min_lcb.x
    
    y_t=wake_model_0(x_bar_t-theta_t)
    
    train_points_1st=np.row_stack([train_points_1st,x_bar_t-theta_t])
    
    train_funval_1st=np.row_stack([train_funval_1st,y_t])
    
    returned_points_1st_exp=returned_points_1st_exp+[x_bar_t]


C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df

In [74]:
# update the hyperparameter of the kernel using the first 90 training samples
def logmarli(x):
    '''
    x is a list of hyperparameters consists of [ell_1 ... ell_D, sqrt(sf2)] 
    and must be positive.
    
    '''
    size=train_points_1st.shape[0]
    log_x=np.log(x)
    par=log_x.tolist()
    K=covSEard(par,train_points_1st)
    K_t=K+0.0001*np.identity(size)
    neg_log_marginal_likelihood=0.5*np.dot(np.dot(train_funval_1st.T,np.linalg.inv(K_t)),train_funval_1st)+0.5*np.log(np.linalg.det(K_t))+250*np.log(2*np.pi)
    
    return neg_log_marginal_likelihood[0,0]

ard=scipy.optimize.minimize(lambda w:logmarli(w), np.array([1.0,1.0,0.6]),method="L-BFGS-B",options = {'disp':True})
ard_param=np.log(ard.x).tolist()

In [83]:
# Last 20 iterations of the first experiment
for t in range(T):
    
    def ucb_min(x):
        
        def k(theta):
            if np.linalg.norm(theta)<=10 and 500<=x[0]-theta[0]<=1000 and 500<=x[1]-theta[1]<=1000:
                return ucb(x-theta,train_points_1st,train_funval_1st)
            else:
                return np.inf
        
        opt=scipy.optimize.differential_evolution(lambda theta: k(theta),bounds=[(-10,10), (-10,10)])
        
        return opt.fun
        
    max_ucb=scipy.optimize.shgo(lambda w:-ucb_min(w), bounds,n=500, iters=5)
    
    x_bar_t=max_ucb.x
    
    def lcb_perturbed(theta):
        
        if np.linalg.norm(theta)<=10 and 500<=x_bar_t[0]-theta[0]<=1000 and 500<=x_bar_t[1]-theta[1]<=1000:
            return lcb(x_bar_t-theta,train_points_1st,train_funval_1st)
        else:
            return np.inf
        
    
    min_lcb=scipy.optimize.differential_evolution(lambda theta: lcb_perturbed(theta),bounds=[(-10,10), (-10,10)])
    
    theta_t=min_lcb.x
    
    y_t=wake_model_0(x_bar_t-theta_t)
    
    train_points_1st=np.row_stack([train_points_1st,x_bar_t-theta_t])
    
    train_funval_1st=np.row_stack([train_funval_1st,y_t])
    
    returned_points_1st_exp=returned_points_1st_exp+[x_bar_t]
    
# Save the output for first experiment.
np.savetxt("wake_model_0_returned_points_1st_exp.txt",returned_points_1st_exp)
np.savetxt("wake_model_0_train_points_1st_exp.txt",train_points_1st)
np.savetxt("wake_model_0_train_funval_1st_exp.txt",train_funval_1st)

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df

In [86]:
#Now do the fourth experiment with another method with theta=0 as shown in the report, for comparing this two methods of training
#sample 10 initial points
np.random.seed(26)
points = []
num_points=10

for i in range(num_points):
    x=np.random.uniform(500,1000)
    y=np.random.uniform(500,1000)
    points = points + [[x,y]]

initial_points=np.asarray(points)
initial_funval=np.asarray([wake_model_0(x) for x in initial_points]).reshape(10,1)

train_points_4th=initial_points
train_funval_4th=initial_funval
bounds=((500, 1000), (500, 1000))

#set the observation noise and the beta parameter
noise=0.0005
beta_sqrt=4
# define mean function, variance function, lcb, ucb of GP model
def mu(x,input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    n,D=input_data_x.shape
    x_list=x.reshape(1,2).tolist()
    K_t=covSEard(ard_param,input_data_x)+noise*np.identity(n)
    mean=np.dot(np.dot(covSEard(ard_param,input_data_x,x_list).T,np.linalg.inv(K_t)),input_data_y)
    
    return mean[0,0]

def sigma_sq(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    n,D=input_data_x.shape
    x_vec=x.reshape(1,2)
    x_list=x_vec.tolist()
    K_t=covSEard(ard_param,input_data_x)+noise*np.identity(n)
    var=covSEard(ard_param,x_vec,x_list)-np.dot(np.dot(covSEard(ard_param,input_data_x,x_list).T,np.linalg.inv(K_t)),covSEard(ard_param,input_data_x,x_list))
    return var[0,0]

def ucb(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    ucb=mu(x,input_data_x, input_data_y)+beta_sqrt*np.sqrt(sigma_sq(x, input_data_x, input_data_y))
    return ucb

def lcb(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    lcb=mu(x,input_data_x, input_data_y)-beta_sqrt*np.sqrt(sigma_sq(x, input_data_x, input_data_y))
    return lcb

# First 60 iterations of the fourth experiment
returned_points_4th_exp=[] # returned points in all iterations
T=60 #number of iteration

for t in range(T):
    
    def ucb_min(x):
        
        def k(theta):
            if np.linalg.norm(theta)<=10 and 500<=x[0]-theta[0]<=1000 and 500<=x[1]-theta[1]<=1000:
                return ucb(x-theta,train_points_4th,train_funval_4th)
            else:
                return np.inf
        
        opt=scipy.optimize.differential_evolution(lambda theta: k(theta),bounds=[(-10,10), (-10,10)])
        
        return opt.fun
        
    max_ucb=scipy.optimize.shgo(lambda w:-ucb_min(w), bounds,n=500, iters=5)
    
    x_bar_t=max_ucb.x
    
    def lcb_perturbed(theta):
        
        if np.linalg.norm(theta)<=10 and 500<=x_bar_t[0]-theta[0]<=1000 and 500<=x_bar_t[1]-theta[1]<=1000:
            return lcb(x_bar_t-theta,train_points_4th,train_funval_4th)
        else:
            return np.inf
        
    
    min_lcb=scipy.optimize.differential_evolution(lambda theta: lcb_perturbed(theta),bounds=[(-10,10), (-10,10)])
    
    theta_t=min_lcb.x
    
    y_t=wake_model_0(x_bar_t-theta_t)
    
    train_points_4th=np.row_stack([train_points_4th,x_bar_t-theta_t])
    
    train_funval_4th=np.row_stack([train_funval_4th,y_t])
    
    returned_points_4th_exp=returned_points_4th_exp+[x_bar_t]

np.savetxt("wake_model_0_returned_points_4th_exp.txt",returned_points_4th_exp)   
np.savetxt("wake_model_0_train_points_4th_exp.txt",train_points_4th) 
np.savetxt("wake_model_0_train_funval_4th_exp.txt",train_funval_4th)

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [10]:
# Wake model with theta=45
def wake_model_45(w):
    '''
    w should be an numpy array or python list of size 2 correspond to the value of
    S_x, S_y in wake_lake function.
    '''
    x,y=w
    return wake_model(x,y,45)

In [11]:
# perturbed wake model with theta=45
def wake_model_45_perturbed(w):
    
    def k(theta):
        if np.linalg.norm(theta)<=10 and 500<=w[0]-theta[0]<=1000 and 500<=w[1]-theta[1]<=1000:
            return wake_model_45(w-theta)
        else:
            return np.inf
        
    opt=scipy.optimize.differential_evolution(lambda theta: k(theta),bounds=[(-10,10), (-10,10)])
    return opt.fun

In [12]:
# Sampling 50 data from wake_model_45
np.random.seed(41)
points = []
num_points=50

for i in range(num_points):
    x=np.random.uniform(500,1000)
    y=np.random.uniform(500,1000)
    points = points + [[x,y]]

sample_points_45=np.asarray(points)
sample_funval_45=np.asarray([wake_model_45(x) for x in sample_points_45]).reshape(50,1)

In [15]:
# Hyperparameter tuning.
def logmarli_45(x):
    '''
    x is a list of hyperparameters consists of [ell_1 ... ell_D, sqrt(sf2)] 
    and must be positive.
    
    '''
    size=sample_points_45.shape[0]
    log_x=np.log(x)
    par=log_x.tolist()
    K=covSEard(par,sample_points_45)
    K_t=K+0.0001*np.identity(size)
    neg_log_marginal_likelihood=0.5*np.dot(np.dot(sample_funval_45.T,np.linalg.inv(K_t)),sample_funval_45)+0.5*np.log(np.linalg.det(K_t))+250*np.log(2*np.pi)
    
    return neg_log_marginal_likelihood[0,0]

ard_45=scipy.optimize.minimize(lambda w:logmarli_45(w), np.array([10.0,10.0,0.6]),method="L-BFGS-B",options = {'disp':True}) #optimized value of ell_1, ell_2 and sqrt(sf2)
ard_param_45=np.log(ard_45.x).tolist()

In [18]:
# Sampling 10 data from wake_model_45
np.random.seed(22)
points = []
num_points=10

for i in range(num_points):
    x=np.random.uniform(500,1000)
    y=np.random.uniform(500,1000)
    points = points + [[x,y]]

initial_points_45=np.asarray(points)
initial_funval_45=np.asarray([wake_model_45(x) for x in initial_points_45]).reshape(10,1)

In [19]:
train_points_6th=initial_points_45
train_funval_6th=initial_funval_45
bounds=((500, 1000), (500, 1000))

#set the observation noise and the beta parameter
noise=0.0005
beta_sqrt=4
# redefine mean function, variance function, lcb, ucb of GP model
def mu(x,input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    n,D=input_data_x.shape
    x_list=x.reshape(1,2).tolist()
    K_t=covSEard(ard_param_45,input_data_x)+noise*np.identity(n)
    mean=np.dot(np.dot(covSEard(ard_param_45,input_data_x,x_list).T,np.linalg.inv(K_t)),input_data_y)
    
    return mean[0,0]

def sigma_sq(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    n,D=input_data_x.shape
    x_vec=x.reshape(1,2)
    x_list=x_vec.tolist()
    K_t=covSEard(ard_param_45,input_data_x)+noise*np.identity(n)
    var=covSEard(ard_param_45,x_vec,x_list)-np.dot(np.dot(covSEard(ard_param_45,input_data_x,x_list).T,np.linalg.inv(K_t)),covSEard(ard_param_45,input_data_x,x_list))
    return var[0,0]

def ucb(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    ucb=mu(x,input_data_x, input_data_y)+beta_sqrt*np.sqrt(sigma_sq(x, input_data_x, input_data_y))
    return ucb

def lcb(x, input_data_x, input_data_y):
    '''
    x is the 1-Dimensional input vector of size D at any points within the domain and input_data_x is the 
    n*D x data set and input_y is the n*1 y value.
    
    '''
    lcb=mu(x,input_data_x, input_data_y)-beta_sqrt*np.sqrt(sigma_sq(x, input_data_x, input_data_y))
    return lcb

# First 60 iterations of the sixth experiment of wake model with theta=45
returned_points_6th_exp=[] # returned points in all iterations
T=60 #number of iteration

for t in range(T):
    
    def ucb_min(x):
        
        def k(theta):
            if np.linalg.norm(theta)<=10 and 500<=x[0]-theta[0]<=1000 and 500<=x[1]-theta[1]<=1000:
                return ucb(x-theta,train_points_6th,train_funval_6th)
            else:
                return np.inf
        
        opt=scipy.optimize.differential_evolution(lambda theta: k(theta),bounds=[(-10,10), (-10,10)])
        
        return opt.fun
        
    max_ucb=scipy.optimize.shgo(lambda w:-ucb_min(w), bounds,n=500, iters=5)
    
    x_bar_t=max_ucb.x
    
    def lcb_perturbed(theta):
        
        if np.linalg.norm(theta)<=10 and 500<=x_bar_t[0]-theta[0]<=1000 and 500<=x_bar_t[1]-theta[1]<=1000:
            return lcb(x_bar_t-theta,train_points_6th,train_funval_6th)
        else:
            return np.inf
        
    
    min_lcb=scipy.optimize.differential_evolution(lambda theta: lcb_perturbed(theta),bounds=[(-10,10), (-10,10)])
    
    theta_t=min_lcb.x
    
    y_t=wake_model_45(x_bar_t-theta_t)
    
    train_points_6th=np.row_stack([train_points_6th,x_bar_t-theta_t])
    
    train_funval_6th=np.row_stack([train_funval_6th,y_t])
    
    returned_points_6th_exp=returned_points_6th_exp+[x_bar_t]

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [21]:
# another 40 iteration for the sixth experiment
for t in range(40):
    
    def ucb_min(x):
        
        def k(theta):
            if np.linalg.norm(theta)<=10 and 500<=x[0]-theta[0]<=1000 and 500<=x[1]-theta[1]<=1000:
                return ucb(x-theta,train_points_6th,train_funval_6th)
            else:
                return np.inf
        
        opt=scipy.optimize.differential_evolution(lambda theta: k(theta),bounds=[(-10,10), (-10,10)])
        
        return opt.fun
        
    max_ucb=scipy.optimize.shgo(lambda w:-ucb_min(w), bounds,n=500, iters=5)
    
    x_bar_t=max_ucb.x
    
    def lcb_perturbed(theta):
        
        if np.linalg.norm(theta)<=10 and 500<=x_bar_t[0]-theta[0]<=1000 and 500<=x_bar_t[1]-theta[1]<=1000:
            return lcb(x_bar_t-theta,train_points_6th,train_funval_6th)
        else:
            return np.inf
        
    
    min_lcb=scipy.optimize.differential_evolution(lambda theta: lcb_perturbed(theta),bounds=[(-10,10), (-10,10)])
    
    theta_t=min_lcb.x
    
    y_t=wake_model_45(x_bar_t-theta_t)
    
    train_points_6th=np.row_stack([train_points_6th,x_bar_t-theta_t])
    
    train_funval_6th=np.row_stack([train_funval_6th,y_t])
    
    returned_points_6th_exp=returned_points_6th_exp+[x_bar_t]
    
np.savetxt("wake_model_45_returned_points_6th_exp.txt",returned_points_6th_exp)   
np.savetxt("wake_model_45_train_points_6th_exp.txt",train_points_6th) 
np.savetxt("wake_model_45_train_funval_6th_exp.txt",train_funval_6th)

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df

In [12]:
# Evaluating wake_model_0_perturbed at (1000, 500)
wake_model_0_perturbed(np.array([1000,500]))

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


0.6866390356070249

In [26]:
# Evaluating wake_model_45_perturbed at (812.47, 562.42)
wake_model_45_perturbed([812.47, 562.42])

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


0.7894147078923758

In [27]:
# Evaluating wake_model_45_perturbed at (999.96, 624.95)
wake_model_45_perturbed([999.96, 624.95])

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


0.779201041608465

In [28]:
# Evaluating wake_model_45_perturbed at (687.57, 999.95)
wake_model_45_perturbed([687.57, 999.95])

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


0.7869442170036708

In [29]:
# Evaluating wake_model_45_perturbed at (906.37, 593.75)
wake_model_45_perturbed([906.37, 593.75])

C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\win\anaconda\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df

0.7856691450791714